In [ ]:
!pip install pybats

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.0/529.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.1/66.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import pybats
from pybats.loss_functions import MAPE
from pybats.analysis import analysis
from pybats.point_forecast import mean

ModuleNotFoundError: ignored

In [ ]:
wheat_prices=[142,151,152,157,162,166,173,183,215,225,280,330,350,360,380,475,510,550,580,610,620,630,640,650,850,1000,1080,1100,1170,1285,1350,1400,1450,1525,1625,1735,1840,1925,1975,2015,2125]
rice_prices=[115,122,132,137,142,146,150,160,185,205,230,270,310,340,360,380,415,440,490,510,530,530,550,560,570,580,745,900,1050,1000,1080,1250,1310,1360,1410,1470,1550,1750,1815,1868,1940,2040]

In [ ]:
len(wheat_prices)

41

In [ ]:
len(rice_prices)

42

In [ ]:
def sampling(sequence, n_steps):
  X, Y = list(), list()
  for i in range(len(sequence)):
    sam = i+ n_steps
    if sam > len(sequence)-1:
      break
    x, y = sequence[i:sam], sequence[sam]
    X.append(x)
    Y.append(y)
  return np.array(X), np.array(Y)

In [ ]:
prices=rice_prices

In [ ]:
n_steps = 5
X, Y = sampling(prices, n_steps)

In [ ]:
X

array([[ 115,  122,  132,  137,  142],
       [ 122,  132,  137,  142,  146],
       [ 132,  137,  142,  146,  150],
       [ 137,  142,  146,  150,  160],
       [ 142,  146,  150,  160,  185],
       [ 146,  150,  160,  185,  205],
       [ 150,  160,  185,  205,  230],
       [ 160,  185,  205,  230,  270],
       [ 185,  205,  230,  270,  310],
       [ 205,  230,  270,  310,  340],
       [ 230,  270,  310,  340,  360],
       [ 270,  310,  340,  360,  380],
       [ 310,  340,  360,  380,  415],
       [ 340,  360,  380,  415,  440],
       [ 360,  380,  415,  440,  490],
       [ 380,  415,  440,  490,  510],
       [ 415,  440,  490,  510,  530],
       [ 440,  490,  510,  530,  530],
       [ 490,  510,  530,  530,  550],
       [ 510,  530,  530,  550,  560],
       [ 530,  530,  550,  560,  570],
       [ 530,  550,  560,  570,  580],
       [ 550,  560,  570,  580,  745],
       [ 560,  570,  580,  745,  900],
       [ 570,  580,  745,  900, 1050],
       [ 580,  745,  900,

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
X = X.reshape((X.shape[0], X.shape[1], 1))

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 50)                10400     
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 10451 (40.82 KB)
Trainable params: 10451 (40.82 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(X, Y, epochs=200, verbose=0)

In [ ]:
x=prices[len(prices)-5:]
x=np.asarray(x)
x=x.reshape((1,n_steps,1))

In [ ]:
rice_future=[]
for i in range(5):
  y=model.predict(x)
  rice_future.append(y)
  x=np.delete(x,0,1)
  x=np.append(x,y)
  x=x.reshape((1,n_steps,1))

1/1 [==============================] - 0s 25ms/step


In [ ]:
rice_future

[array([[2206.1577]], dtype=float32),
 array([[2357.1836]], dtype=float32),
 array([[2432.185]], dtype=float32),
 array([[2534.6223]], dtype=float32),
 array([[2680.2983]], dtype=float32)]

In [ ]:
import pickle
with open('rice_future.pkl', 'wb') as f:
    pickle.dump(rice_future, f)